In [2]:
import hydra
import torch
import torchaudio

exp_cfg = "train_musdb_conditional"
ckpt_path = "../ckpts/musdb-conditional_epoch=423.ckpt"

with hydra.initialize(config_path="..", version_base=None):
    cond_cfg = hydra.compose(config_name="config", overrides=[f'exp={exp_cfg}'])
model = hydra.utils.instantiate(cond_cfg["model"])

ckpt = torch.load(ckpt_path, map_location="cpu")
model.load_state_dict(ckpt['state_dict'], strict=False)
model = model.cuda()

In [ ]:
y, sr = torchaudio.load("in.wav") # load you audio input
prompt = "in: other_1, vocals_1; out: vocals_1"

assert sr == 16000

y = torch.clip(y, -1, 1)
y_melspec = model.stft.cpu().mel_spectrogram(y.cpu())[0]
y_latents = model.vae.encode(y_melspec.cuda().permute(0, 2, 1).unsqueeze(1)).latent_dist.sample()
y_latents = y_latents * model.vae.config.scaling_factor

samples = model.pipeline([prompt], num_inference_steps=150,
         guidance_scale=1.0, audio_length_in_s=10.23, controlnet_cond=y_latents.cuda()).audios
torchaudio.save(f"out.wav", torch.tensor(samples[0]).unsqueeze(0), sample_rate=sr)